In [1]:
# %pip install nest_asyncio

import asyncio

import nest_asyncio


nest_asyncio.apply()

In [2]:
# from airflow.decorators import task
import requests
import json
import os
import asyncio
from aiohttp import ClientSession

import pandas as pd
import numpy as np
import itertools
from datetime import datetime
from typing import Tuple
loop = asyncio.ProactorEventLoop()
asyncio.set_event_loop(loop)

In [3]:
MAXIMUM_CONCURRENT_REQUESTS = 200

riot_tokens = [
    "RGAPI-d93ad850-d1e8-404a-a9f7-db6ba9a6016b", #PBE
    "RGAPI-58b977e3-1fe9-4eef-a1b9-8cf817e627fa", #NA
    "RGAPI-da3e753f-9145-4edc-8be5-54dce7e81e5c", #EUW
    "RGAPI-38d5ab33-910b-492e-ac00-bb5f086e03e8", #EUNE
    "RGAPI-07a5a53e-bbd1-44d2-9d5d-2c0f08b476d8", #MAIN
]
region_groupings = {
    "br1":"americas",
    "eun1":"europe",
    "euw1":"europe",
    "jp1":"asia",
    "kr":"asia",
    "la1":"americas",
    "la2":"americas",
    "na1":"americas",
    "oc1":"sea",
    "tr1":"europe",
    "ru":"europe",
    "ph2":"sea",
    "sg2":"sea",
    "th2":"sea",
    "tw2":"sea",
    "vn2":"sea",
}

In [4]:
start_date = datetime.now()
count = 0
rateLimitedCount = 0
async def api_request(url, riot_token: str, fromRateLimited = 0) -> dict:
    header = {
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://developer.riotgames.com",
        "X-Riot-Token": riot_token,
    }
    async with ClientSession() as session:
        async with session.get(url, headers=header) as response:
            global rateLimitedCount
            global count 
            count+=1
            if fromRateLimited :
                rateLimitedCount -=1
                print(f"No longer Rate limited count: {rateLimitedCount}")

            print(f"Took {datetime.now() - start_date} to complete {count} requests on {url} and API token {riot_token}")
            if response.status == 429:
                retry_after = int(response.headers.get('Retry-After', '1'))
                print("Rate limited, retrying after " + str(retry_after) + " seconds.")

                rateLimitedCount +=1
                print("Rate limited count: " + str(rateLimitedCount))

                await asyncio.sleep(retry_after)
                return await api_request(url, riot_token,1)
            if response.status != 200:
                print(f"Error {response.status} on {url} with API {riot_token}")
            # assert response.status == 200
            return await response.json()
            

In [5]:
# @task(task_id="multiple_matches_ingest")
async def matches_request(matches: list[str]) -> list[dict]:
    requests_list = []
    for match in matches:
        print("Fetching match: " + match)
        region = region_groupings[match.split('_')[0].lower()]
        match_uri = f"https://{region}.api.riotgames.com/lol/match/v5/matches/{match}"
        requests_list.append(api_request(match_uri, np.random.choice(riot_tokens,1)[0]))

    semaphore = asyncio.Semaphore(MAXIMUM_CONCURRENT_REQUESTS)
    async def semaphored_request(request):
        async with semaphore:
            return await request
    return await asyncio.gather(*(semaphored_request(request) for request in requests_list))

async def players_matches_request(player_uuids : list[tuple], match_count : int ) -> list[dict]:
    requests_list = []
    for puuid, region_group, riot_token in player_uuids:
        request = api_request(f"https://{region_group}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count={match_count}", riot_token)
        requests_list.append(request)

    semaphore = asyncio.Semaphore(MAXIMUM_CONCURRENT_REQUESTS)
    async def semaphored_request(request):
        async with semaphore:
            return await request
        
    return await asyncio.gather(*(semaphored_request(request) for request in requests_list))
    # return await asyncio.gather(*requests_list)
    
async def league_entries_request(queue: str, tier :str, division:str, page:int , region : str, riot_token : str) -> list[dict]:
    request = api_request(f"https://{region}.api.riotgames.com/lol/league-exp/v4/entries/{queue}/{tier}/{division}?page={page}", riot_token)
    return await asyncio.gather(*[request])

async def top_league_entries_request(queue : str, tier:str, region : str, riot_token : str) -> list[dict]:
    request = api_request(f"https://{region}.api.riotgames.com/lol/league/v4/{tier}/by-queue/{queue}", riot_token)
    return await asyncio.gather(*[request])

async def players_request(players : list[tuple]): #By SummonnerID -> "id" in JSON file
    requests_list = []

    for region, summonerID, riot_token in players:
        requests_list.append(api_request(f"https://{region}.api.riotgames.com/lol/summoner/v4/summoners/{summonerID}", str(riot_token)))
    
    semaphore = asyncio.Semaphore(MAXIMUM_CONCURRENT_REQUESTS)
    async def semaphored_request(request):
        async with semaphore:
            return await request
    
    # return await asyncio.gather(*requests_list)
    return await asyncio.gather(*(semaphored_request(request) for request in requests_list))

In [6]:
# @task(task_id="multiple_matches_ingest")
def matches_ingest(matches: list[str], limit = None) -> list[dict]:
    if limit is None:
        limit = len(matches)
    
    return asyncio.run(matches_request(itertools.islice(matches, 0, limit-1)))

# @task(task_id="players_matches_ingest")
def players_matches_ingest(player_uuids : list[tuple], match_count : int = 20) -> list[dict]:
    return asyncio.run(players_matches_request(player_uuids, match_count))
    
# @task(task_id="league_entries_ingest")
def league_entries_ingest(queue: str, tier :str, division:str, region, riot_token : str, page:int = 1) -> list[dict]:
    return asyncio.run(league_entries_request(queue, tier, division, page, region, riot_token))[0]

# @task(task_id="top_league_entries_ingest")
def top_league_entries_ingest(queue: str, tier:str,region, riot_token : str) -> dict:
    return asyncio.run(top_league_entries_request(queue, tier, region,riot_token))[0]

# @task(task_id="player_ingest")
def players_ingest(players : list[tuple]) -> list[dict]:
    return asyncio.run(players_request(players))



### Clean DataFrame

In [7]:
# @task(task_id="clean_dataframe_dtypes")
def clean_dataframe_dtypes(df: pd.DataFrame, categorical_columns = [], int_columns = [], datetime_columns = []) -> pd.DataFrame:
    for column in categorical_columns:
        df[column] = df[column].astype("category")
    for column in int_columns:
        df[column] = df[column].astype("int")
    for column in datetime_columns:
        df[column] = pd.to_datetime(df[column])
    return df

# @task(task_id="reorder_coluns")
def reorder_columns(df: pd.DataFrame, ordered_columns: list[str]) -> pd.DataFrame:
    return df.reindex(columns=ordered_columns)

# @task(task_id="clean_dataframe")
def clean_dataframe(df: pd.DataFrame, categorical_columns = [], int_columns = [], datetime_columns = [], ordered_columns = []) -> pd.DataFrame:
    df = clean_dataframe_dtypes(df, categorical_columns, int_columns, datetime_columns)
    if len(ordered_columns) > 0:
        df = reorder_columns(df, ordered_columns)
    return df

### Saving output to csv

In [8]:
# @task(task_id="save_to_csv")
def save_to_csv(df: pd.DataFrame, path: str, index: bool = False) -> None:
    """
        Note: Saving to CSV files are only temporary and will be removed after the entire piepline is completed.
    """
    if(os.path.exists(path)):
        df.to_csv(path, index=index, mode='a', header=False)
    else:
        df.to_csv(path, index=index, mode='w', header=True)

### League Entries

In [9]:
#League Infos and Player League Infos is separated

# @task(task_id="top_league_entries")
def top_league_entries(queue: str, tier :str, region) -> Tuple[pd.DataFrame, pd.DataFrame]:
    riot_token : str = np.random.choice(riot_tokens)
    league_entries_raw : dict = top_league_entries_ingest(queue, tier, region, riot_token)

    if not league_entries_raw:
        return pd.DataFrame(), pd.DataFrame()
    
    current_time = datetime.now()

    player_league_infos = pd.DataFrame(league_entries_raw["entries"])
    player_league_infos["leagueId"] = league_entries_raw.get("leagueId")
    player_league_infos["last_updated"] = datetime.now()
    player_league_infos["riot_token"] = riot_token
    player_league_infos["region"] = region
    # player_league_infos = reorder_columns(player_league_infos, ordered_columns= ["leagueId","region","summonerId","summonerName","leaguePoints","rank","wins","losses","veteran","inactive","freshBlood","hotStreak","last_updated","riot_token"])

    del league_entries_raw["entries"]
    league_infos = pd.DataFrame([league_entries_raw])
    league_infos["last_updated"] = datetime.now()
    league_infos["queue"] = queue
    league_infos["region"] = region
    league_infos["division"] = 'I'
    league_infos["riot_token"] = riot_token
    # league_infos = reorder_columns(league_infos, ordered_columns= ["leagueId","region","queue","tier","division","name","last_updated","riot_token"])
    return league_infos, player_league_infos

# @task(task_id="league_entries")
def normal_league_entries(queue: str, tier :str, division:str, region : str, pages : int) -> Tuple[pd.DataFrame, pd.DataFrame]:
    player_leauge_infos_list = []
    league_infos_list = []
    for page in range(1, pages+1):
        riot_token : str = np.random.choice(riot_tokens)
     
        league_entries_raw : dict = pd.DataFrame.from_dict(league_entries_ingest(queue, tier, division, region, riot_token, page))

        if league_entries_raw.empty:
            return pd.DataFrame(), pd.DataFrame()

        player_league_infos = league_entries_raw.loc[:,['leagueId','summonerId','summonerName','leaguePoints','rank','wins','losses','veteran','inactive','freshBlood','hotStreak']]
        player_league_infos["last_updated"] = datetime.now()
        player_league_infos["riot_token"] = riot_token
        player_league_infos["region"] = region
        player_leauge_infos_list.append(player_league_infos)
        # player_league_infos = reorder_columns(player_league_infos,["leagueId","region","summonerId","summonerName","leaguePoints","rank","wins","losses","veteran","inactive","freshBlood","hotStreak","last_updated","riot_token"])

        league_infos = league_entries_raw.loc[:,['leagueId','tier']]
        league_infos["region"] = region
        league_infos["queue"] = queue
        league_infos["division"] = league_entries_raw["rank"]
        league_infos["last_updated"] = datetime.now()
        league_infos["riot_token"] = riot_token
        # league_infos = reorder_columns(league_infos, ordered_columns= ["leagueId","region","queue","tier","division","name","last_updated","riot_token"])
        league_infos.drop_duplicates(subset=["leagueId"], inplace=True)
        league_infos_list.append(league_infos)

    return pd.concat(league_infos_list, axis=0), pd.concat(player_leauge_infos_list,axis=0)

# @task(task_id="all_league_entries")
def league_entries(queue : str,
                   tier : str, 
                   division: str = None,
                   regions: list[str] = ["ph2", "eun1", "euw1", "jp1", "kr", "la1", "la2", "na1", "oc1", "ru", "sg2", "th2", "tr1", "tw2", "vn2","br1"],
                   pages : int = 1):
    """
        Main function for ingestion of data for leagues.
        Args:
            queue::str:
                The queue type of the league.  All valid values accepted according to API: RANK_FLEX_SR, RANK_FLEX_TT, RANKED_SOLO_5x5. Note that RANK_FLEX_TT is deprecated.
            tier::str:
                The league tier.  The following values are accepted: challengerleagues, grandmasterleagues, masterleagues, DIAMOND, PLATINUM, GOLD, SILVER, BRONZE, IRON
            division::str:
                The league division.  All valid values accepted according to API: I, II, III, IV.  Challenger to Master divisions only have one division.
            region::str:
                The region of the league. Default: All regions.  All valid values accepted according to API: ph2, eun1, euw1, jp1, kr, la1, la2, na1, oc1, ru, sg2, th2, tr1, tw2, vn2
            pages::int:
                The number of pages to be ingested.  Each page contains >20 entries and varies per region. Challenger to Master divisions always show the complete list (only 1 page).  
        Returns:
            None
    """
    league_infos_list = []
    if tier in ["challengerleagues", "grandmasterleagues", "masterleagues"]:
        league_infos_list = [top_league_entries(queue=queue, tier=tier, region=region) for region in regions]
    else:
        if division is None:
            raise ValueError("Division must be specified for non-Challenger to Master divisions.")
        league_infos_list = [normal_league_entries(queue=queue, tier=tier, division=division, region=region, pages=pages) for region in regions]

    league_infos_list = list(zip(*league_infos_list))
    return pd.concat(league_infos_list[0],axis=0), pd.concat(league_infos_list[1],axis=0)


### Player Details

In [11]:
league_info, player_league_info = league_entries(queue="RANKED_SOLO_5x5", tier="challengerleagues", division='I', pages=1)
player_ids = list(player_league_info[["region","summonerId","riot_token"]].itertuples(index=False, name=None))
player_info = players_ingest(player_ids)

Took 0:00:38.749173 to complete 2 requests on https://ph2.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5 and API token RGAPI-d93ad850-d1e8-404a-a9f7-db6ba9a6016b
Error 403 on https://ph2.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5 with API RGAPI-d93ad850-d1e8-404a-a9f7-db6ba9a6016b


KeyError: 'entries'

In [ ]:
accounts_raw = pd.DataFrame(player_info).merge(player_league_info, left_on='id', right_on = 'summonerId', how='right')

accounts_raw["region_group"] = accounts_raw["region"].map(region_groupings)
assert accounts_raw["region_group"].isna().sum() == 0

# display(accounts_raw["region"].unique())
match_history_raw = players_matches_ingest(list(accounts_raw[['puuid','region_group','riot_token']].itertuples(index=False, name=None)))

Took 0:33:17.162338 to complete 15403 requests on https://sea.api.riotgames.com/lol/match/v5/matches/by-puuid/9yLQPygucQUizvWGrHIGOTJQEdqIwELggpXo9c1kyDdRPqge31MBjaLzBdjCThG-71_YA_CUk0eMzw/ids?start=0&count=20 and API token RGAPI-d93ad850-d1e8-404a-a9f7-db6ba9a6016b
Rate limited, retrying after 2 seconds.
Rate limited count: 11
Took 0:33:17.176323 to complete 15404 requests on https://sea.api.riotgames.com/lol/match/v5/matches/by-puuid/3LkIfyIhZP1_g57-8iDKjRRo3zhcB-ZW-178uBsez9XJUog5FWK2oXdfRCyeL3SC1_u_-KxohOFiCw/ids?start=0&count=20 and API token RGAPI-d93ad850-d1e8-404a-a9f7-db6ba9a6016b
Rate limited, retrying after 2 seconds.
Rate limited count: 12
Took 0:33:17.183480 to complete 15405 requests on https://sea.api.riotgames.com/lol/match/v5/matches/by-puuid/UQpGAuaGvo2k8AQdVoIX9CKn7sKJzG60cVYzqE-0huisuGM_ZpK-y25lEhVziLTqL2lvnqXByrXTsA/ids?start=0&count=20 and API token RGAPI-d93ad850-d1e8-404a-a9f7-db6ba9a6016b
Rate limited, retrying after 2 seconds.
Rate limited count: 13
Took 0:33:

In [ ]:

match_history_list = set(itertools.chain.from_iterable(match_history_raw))
raw_match_history_details = matches_ingest(match_history_list, 3000)


Fetching match: RU_435038708
Fetching match: NA1_4602846851
Fetching match: LA2_1280731187
Fetching match: LA2_1278214825
Fetching match: EUW1_6328908072
Fetching match: EUN1_3334438846
Fetching match: TR1_1390568793
Fetching match: BR1_2698384332
Fetching match: VN2_53993533
Fetching match: NA1_4608264757
Fetching match: TR1_1392075600
Fetching match: VN2_53999277
Fetching match: BR1_2690624675
Fetching match: EUN1_3334984119
Fetching match: PH2_8245658
Fetching match: EUW1_6313690293
Fetching match: BR1_2700970157
Fetching match: BR1_2700839663
Fetching match: JP1_390433512
Fetching match: EUN1_3335595354
Fetching match: LA2_1276408333
Fetching match: LA1_1369760092
Fetching match: NA1_4595820748
Fetching match: LA2_1280388589
Fetching match: VN2_53343253
Fetching match: TR1_1391759686
Fetching match: PH2_7950648
Fetching match: KR_6416394663
Fetching match: KR_6418924237
Fetching match: LA1_1369570135
Fetching match: KR_6413974868
Fetching match: EUN1_3336947456
Fetching match: NA1_

### Match Details   

In [ ]:
# @task(task_id="match_general_info")
def get_match_general_info(raw_match: dict) -> dict:
    match = {}

    match['matchId'] = raw_match['metadata']['matchId']
    keys_to_exlude = {
        "participants",
        "teams",
    }
    match |= { key:raw_match["metadata"][key] for key in set(set(raw_match["metadata"].keys()) - keys_to_exlude)}
    match |= { key:raw_match["info"][key] for key in set(set(raw_match["info"].keys()) - keys_to_exlude)}
    return match

# @task(task_id = "match_players_info")
def get_match_players_info(raw_match: dict) -> list[dict]:
    if raw_match is None:
        print("Match none detected")
        return {}
    
    keys_initial_exclude = {
        "challenges",
        "perks",
    }

    raw_participants_details  : list[dict] = raw_match["info"]["participants"]

    final_participants_details : list[dict] = []
    
    for raw_participant in raw_participants_details:
        participant = {}
        
        #Normal Keys
        participant |= { key:raw_participant[key] for key in set(set(raw_participant.keys()) - keys_initial_exclude)}

        #Challenges DTO flattened
        if "challenges" in raw_participant.keys():
            participant |= { f"challenges_{key}":raw_participant["challenges"][key] for key in raw_participant["challenges"].keys()}

        #Runes DTO stat perks flattened
        participant |= { f"statPerks_{key}": raw_participant["perks"]["statPerks"][key] for key in raw_participant["perks"]["statPerks"].keys() }
        
        #Runes DTO styles flattened
        for style in raw_participant["perks"]["styles"]:
            temp = {}

            style_type = style['description']
            temp[f"{ style_type }_id"] = style["style"]
            
            #PerkStyleSelectionDto
            for index, perk in enumerate(style["selections"]):
                temp[f"{style_type}_{index}_id"] = perk["perk"]
                temp[f"{style_type}_{index}_var1"] = perk["var1"]
                temp[f"{style_type}_{index}_var2"] = perk["var2"]
                temp[f"{style_type}_{index}_var3"] = perk["var3"]
            
            participant |= temp

        final_participants_details.append(participant)
    
    return final_participants_details

def get_match_teams_info(raw_match: dict) -> dict:
    teams_info_raw = raw_match["info"]["teams"]
    teams_info = {}

    teams_info["matchId"] = raw_match["metadata"]["matchId"]
    for team in teams_info_raw:
        team_name = "blue" if team["teamId"] == 100 else "red"
        for bans in team["bans"]:
            teams_info[f"bans_turn_{bans['pickTurn']}"] = bans["championId"]
        for objective in team["objectives"].keys():
            teams_info[f"{team_name}_{objective}_isFirstKill"] = team["objectives"][objective]["first"]
            teams_info[f"{team_name}_{objective}_kills"] = team["objectives"][objective]["kills"]
        teams_info[f"{team_name}_win"] = team["win"]
    return teams_info




In [ ]:
match_general_info = pd.DataFrame([get_match_general_info(match) for match in raw_match_history_details])
display(match_general_info)

match_player_info = itertools.chain.from_iterable([get_match_players_info(match) for match in raw_match_history_details])
match_player_info = pd.DataFrame(match_player_info)
display(match_player_info)

match_teams_info = pd.DataFrame([get_match_teams_info(match) for match in raw_match_history_details])
display(match_teams_info)

,matchId,dataVersion,gameCreation,mapId,queueId,gameEndTimestamp,gameMode,tournamentCode,gameType,gameDuration,gameId,gameVersion,gameStartTimestamp,platformId,gameName
0,RU_435038708,2,1679584376474,11,400,1679586359484,CLASSIC,,MATCHED_GAME,1913,435038708,13.6.499.913,1679584446394,RU,teambuilder-match-435038708
1,NA1_4602846851,2,1678945422148,11,420,1678946408901,CLASSIC,,MATCHED_GAME,967,4602846851,13.5.495.8836,1678945441738,NA1,teambuilder-match-4602846851
2,LA2_1280731187,2,1679609367071,11,420,1679611060907,CLASSIC,,MATCHED_GAME,1622,1280731187,13.6.499.913,1679609438176,LA2,teambuilder-match-1280731187
3,LA2_1278214825,2,1678899109847,11,420,1678901093213,CLASSIC,,MATCHED_GAME,1953,1278214825,13.5.495.8836,1678899139180,LA2,teambuilder-match-1278214825
4,EUW1_6328908072,2,1679614505176,12,450,1679616162245,ARAM,,MATCHED_GAME,1565,6328908072,13.6.499.913,1679614596445,EUW1,teambuilder-match-6328908072
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2994,NA1_4609777684,2,1679627296862,12,450,1679628089137,ARAM,,MATCHED_GAME,740,4609777684,13.6.499.7165,1679627348897,NA1,teambuilder-match-4609777684
2995,TW2_46472377,2,1679123482156,12,450,1679125042387,ARAM,,MATCHED_GAME,1512,46472377,13.5.495.8836,1679123530393,TW2,teambuilder-match-46472377
2996,EUN1_3334925679,2,1679096478343,11,440,1679098239638,CLASSIC,,MATCHED_GAME,1651,3334925679,13.5.495.8836,1679096588234,EUN1,teambuilder-match-3334925679
2997,LA1_1368556230,2,1679333125979,11,420,1679335305119,CLASSIC,,MATCHED_GAME,2142,1368556230,13.5.495.8836,1679333162743,LA1,teambuilder-match-1368556230


,inhibitorTakedowns,nexusLost,totalDamageTaken,firstBloodAssist,trueDamageDealt,riotIdTagline,turretTakedowns,enemyMissingPings,damageDealtToObjectives,profileIcon,...,challenges_firstTurretKilledTime,challenges_highestCrowdControlScore,challenges_thirdInhibitorDestroyedTime,challenges_shortestTimeToAceFromFirstTakedown,challenges_fasterSupportQuestCompletion,challenges_highestWardKills,challenges_baronBuffGoldAdvantageOverThreshold,challenges_hadAfkTeammate,challenges_teleportTakedowns,challenges_mejaisFullStackInTime
0,0,1,26106,False,3284,,0,3,7494,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,1,42981,False,33156,,0,1,17259,3778,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,1,36150,True,13849,,2,6,17897,3838,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,1,28864,False,37985,,0,7,1046,5182,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,1,24569,False,994,,1,0,517,3836,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29961,1,0,37032,False,6676,,6,2,10344,4793,...,988.011877,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29962,0,0,49686,False,4245,,4,9,5544,4405,...,988.011877,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29963,1,0,91610,False,106858,,5,3,23393,5681,...,988.011877,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29964,0,0,79597,False,59228,,1,16,53644,3633,...,988.011877,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,matchId,bans_turn_1,bans_turn_2,bans_turn_3,bans_turn_4,bans_turn_5,blue_baron_isFirstKill,blue_baron_kills,blue_champion_isFirstKill,blue_champion_kills,...,red_champion_kills,red_dragon_isFirstKill,red_dragon_kills,red_inhibitor_isFirstKill,red_inhibitor_kills,red_riftHerald_isFirstKill,red_riftHerald_kills,red_tower_isFirstKill,red_tower_kills,red_win
0,RU_435038708,136.0,238.0,25.0,8.0,119.0,True,1,True,36,...,42,True,5,True,3,False,1,True,11,True
1,NA1_4602846851,350.0,119.0,497.0,412.0,29.0,False,0,True,31,...,7,False,0,False,0,False,0,False,1,False
2,LA2_1280731187,38.0,64.0,119.0,58.0,59.0,False,0,True,29,...,43,False,2,True,1,True,2,True,8,True
3,LA2_1278214825,221.0,887.0,24.0,350.0,134.0,True,1,True,34,...,20,False,1,False,0,True,1,True,5,False
4,EUW1_6328908072,NaN,NaN,NaN,NaN,NaN,False,0,False,73,...,81,False,0,True,2,False,0,True,4,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2994,NA1_4609777684,NaN,NaN,NaN,NaN,NaN,False,0,True,39,...,30,False,0,False,0,False,0,False,0,False
2995,TW2_46472377,NaN,NaN,NaN,NaN,NaN,False,0,False,93,...,76,False,0,True,1,False,0,True,4,False
2996,EUN1_3334925679,498.0,106.0,202.0,420.0,7.0,False,0,False,30,...,38,True,2,True,3,True,2,True,9,True
2997,LA1_1368556230,25.0,117.0,157.0,136.0,77.0,True,1,False,36,...,50,True,4,False,2,True,2,True,11,True
